In [1]:
import os
import pandas as pd

# Functions to compute the numerical results

In [2]:
def load_metrics(result_folder, split):
    if "LFLR" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    elif "LFCO" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    else:
        metrics_file = os.path.join(result_folder, f"split_{split}", f"metrics_split{split}.csv")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"File not found: {metrics_file}")
    
    return pd.read_csv(metrics_file)

# Function to calculate the mean and standard deviation of metrics
def calculate_statistics(result_folder, splits=[1, 2, 3]):
    all_metrics = []

    # Load metrics from all splits
    for split in splits:
        split_metrics = load_metrics(result_folder, split)
        all_metrics.append(split_metrics)

    # Concatenate all split metrics into a DataFrame
    metrics_df = pd.concat(all_metrics)

    # Calculate mean and standard deviation for each metric
    statistics = {
        "mean": metrics_df.mean() * 100,
        "std_dev": metrics_df.std() * 100
    }

    return statistics

def get_valid_subfolders(directory):
    return sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f)) and not f.startswith('.')])

# Results

In [3]:
models = ["./GRU", './FHSI']

results_list = []

for model in models:
    result_subfolders = get_valid_subfolders(model)
    
    for result_folder in result_subfolders:
        result_path = os.path.join(model, result_folder)

        try:
            statistics = calculate_statistics(result_path)
            mean = statistics["mean"]
            std_dev = statistics["std_dev"]
            row = {
                "Model": f"{os.path.basename(model)}_{result_folder}",
                "Accuracy": f"{mean['accuracy']:.2f} ± {std_dev['accuracy']:.2f}",
                "Specificity": f"{mean['specificity']:.2f} ± {std_dev['specificity']:.2f}",
                "Sensitivity": f"{mean['recall']:.2f} ± {std_dev['recall']:.2f}",
                "ROC AUC": f"{mean['roc_auc']:.2f} ± {std_dev['roc_auc']:.2f}"
            }
            
            results_list.append(row)
        
        except FileNotFoundError as e:
            print(e)

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by="Model").reset_index(drop=True)

In [4]:
results_df

,Model,Accuracy,Specificity,Sensitivity,ROC AUC
0,FHSI_Results_FHSI-4MTS-3Features,70.94 ± 0.50,74.17 ± 1.68,56.15 ± 6.16,70.11 ± 0.91
1,FHSI_Results_FHSI-5MTS-3Features,67.98 ± 2.94,68.92 ± 4.25,63.95 ± 3.45,70.82 ± 1.31
2,GRU_Results_GRU-3MTS,66.70 ± 3.15,68.05 ± 4.18,60.05 ± 2.52,69.51 ± 1.22


In [5]:
results_df['Accuracy'].max()

'70.94 ± 0.50'

In [6]:
results_df['Specificity'].max()

'74.17 ± 1.68'

In [7]:
results_df['Sensitivity'].max()

'63.95 ± 3.45'

In [8]:
results_df['ROC AUC'].max()

'70.82 ± 1.31'

In [9]:
for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']:
    results_df[[f'{col}_Mean', f'{col}_StdDev']] = results_df[col].str.split(' ± ', expand=True).astype(float)

mean_values = results_df[[f'{col}_Mean' for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']]].mean()
mean_stddevs = results_df[[f'{col}_StdDev' for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']]].mean()

print("Mean of the values:")
print(mean_values)
print("\nMean of the standard deviations:")
print(mean_stddevs)

Mean of the values:
Accuracy_Mean       68.540000
Specificity_Mean    70.380000
Sensitivity_Mean    60.050000
ROC AUC_Mean        70.146667
dtype: float64

Mean of the standard deviations:
Accuracy_StdDev       2.196667
Specificity_StdDev    3.370000
Sensitivity_StdDev    4.043333
ROC AUC_StdDev        1.146667
dtype: float64
